In [1]:
# Pradėta 2024-11-21_ObjectProperties.ipynb
# Tęsinys. Čia 2024-12-04...

import pandas as pd
import time
import sqlite3
import random

from bs4 import BeautifulSoup

# Selenium
import selenium
from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium import webdriver

from my_lib.properties_of_7 import properties_of
from my_lib.my_db_functions import df_to_sqlite

In [36]:
# Nustatymai, parametrai:

item_type = 'mobilieji-telefonai'
# 'dronai', 'televizoriai', 'saldytuvai', 'televizoriai'
item_params = 'f.s-5g_rysys=Yra&f.s-gamintojas=Samsung&sort=-price&'

table_version = '_8'
item_producer = 'samsung'

common_prop = {'producer':'samsung','5g':True}

url = f'https://www.varle.lt/{item_type}/?{item_params}'
print(url)

https://www.varle.lt/mobilieji-telefonai/?f.s-5g_rysys=Yra&f.s-gamintojas=Samsung&sort=-price&


In [3]:
# Šis fragmentas automatiškai išvalo 'items_df' sukauptus duomenis,
# Pasirengta naujų duomenų įrašymui.

items_df = pd.DataFrame(columns=('id','price','url','price_in_cents','producer','5g'))
# čia bus šios programos rezultatas

In [ ]:
# SCRAP

# Atidaro Chrome naršyklę,
# reikia patvirtinti "Leisti visus slapukus"!
# Šis fragmentas automatiškai panaikina „draiver“.

opcijos = Options()
opcijos.add_argument('--incognito')
opcijos.add_argument('--ignore-certificate-errors')
opcijos.add_argument('--ignore-ssl-errors')
driver = webdriver.Chrome(options=opcijos)

# url = f'https://www.varle.lt/{item_type}/?{item_params}' # iškeliamas į pradžią?

driver.get(url)
# time.sleep(3)
source = driver.page_source
print(f'First page: {url}')

bs = BeautifulSoup(source,'html.parser')
pages_numbers = bs.find('div',{'class':'PAGINATION'}).find_all('li',{'class':'number'})
total_pages = int(pages_numbers[-1].text.strip())
print(f'Total pages: {total_pages}')

In [ ]:
# SCRAP

# Toliau analizuojamas prekių sąrašo html iš bs=BeautifulSoup().

items_blocks = bs.find_all('div',{'class':'GRID_ITEM'})
for item_block in items_blocks:

    item_href = item_block.find('div',{'class':'product-title'}).find('a')['href']
    item_url = 'https://www.varle.lt'+ item_href
    item_id = item_href.split('-')[-1].split('.')[0]

    price_container = item_block.find('div',{'class':'price-container'})
    price_value = price_container.find('span',{'class':'price-value'}).getText()
    item_price = ''.join(price_value.split())

    items_df.loc[len(items_df)] = {'id':int(item_id),'price':item_price,'url':item_url}

print(f'DF items: {len(items_df)}.')

In [ ]:
# SCRAP

# Perskaitomi kiti prekių sąrašo puslapiai,
# po to įrašoma į db

import random

# for page_number in range(2,total_pages+1):
for page_number in range(2,total_pages+1):

    url = f'https://www.varle.lt/{item_type}/?{item_params}p={page_number}'
    driver.get(url)
    time.sleep(random.randint(25,75))
    source = driver.page_source
    print(f'Prskaityta {url}')

    bs = BeautifulSoup(source,'html.parser')

    items_blocks = bs.find_all('div',{'class':'GRID_ITEM'})
    for item_block in items_blocks:
        item_href = item_block.find('div',{'class':'product-title'}).find('a')['href']
        item_url = 'https://www.varle.lt'+item_href
        item_id = item_href.split('-')[-1].split('.')[0]

        price_container = item_block.find('div',{'class':'price-container'})
        price_value = price_container.find('span',{'class':'price-value'}).getText()
        item_price = ''.join(price_value.split())

        # items_df.loc[len(items_df)] = {'id':int(item_id),'price':item_price,'url':item_url}
        items_df.loc[len(items_df)] = {'id':int(item_id),'price':item_price,'url':item_url}

    items_total = len(items_df)
    items_df.drop_duplicates(subset='id',keep='last',inplace=True)
    print(f'Get {items_total} items. Unique {len(items_df)} items.')

# driver.close() # Uždaro Chrome puslapį.

In [ ]:
# Pridedamas kainos stulpelis centais (int)

import re

def price_in_cents(price:str) -> int:
    # Pašalina visus simbolius, išskyrus skaičius, kablelį ir tašką:
    cleaned = re.sub(r'[^\d.,]', '', price).replace(',', '.').replace(' ', '')
    return int(float(cleaned)*100)

items_df['price_in_cents'] = items_df['price'].apply(price_in_cents)

items_df['producer'] = 'samsung'
items_df['5g'] = True

In [4]:
items_df

,id,price,url,price_in_cents,producer,5g


In [37]:
f'{item_type}{table_version}'

'mobilieji-telefonai_8'

In [38]:
print('Save DF to SQLite3. Plati lentelė, ne properties_of()')

df_to_sqlite(items_df,"./my_scrap_working_data.db",table_name=f'{item_type}{table_version}')

Save DF to SQLite3. Plati lentelė, ne properties_of()
Lentelė 'mobilieji-telefonai_8' sėkmingai įrašyta į DB.


In [40]:
print('Iš DF kuriamas properties_of() ir įrašomas į DB.')

mobilieji_telefonai = properties_of( "mobilieji_telefonai",
    engine="sqlite_file", db_path="./my_scrap_prop_of.db")

mobilieji_telefonai.import_from_df(items_df,id='object_id')

Iš DF kuriamas properties_of() ir įrašomas į DB.


In [41]:
items_df

property_id,object_id,5g,price,price_in_cents,producer,url,type,create,update
0,14511169,1,729.99€,72999,samsung,https://www.varle.lt/mobilieji-telefonai/samsu...,mobilieji-telefonai,1733334394,1733334394
1,14522358,1,709.99€,70999,samsung,https://www.varle.lt/mobilieji-telefonai/isman...,mobilieji-telefonai,1733334394,1733334394
2,15087466,1,439.99€,43999,samsung,https://www.varle.lt/mobilieji-telefonai/samsu...,mobilieji-telefonai,1733334394,1733334394
3,15124168,1,511.38€,51138,samsung,https://www.varle.lt/mobilieji-telefonai/samsu...,mobilieji-telefonai,1733334394,1733334394
4,15124169,1,689.99€,68999,samsung,https://www.varle.lt/mobilieji-telefonai/samsu...,mobilieji-telefonai,1733334394,1733334394
...,...,...,...,...,...,...,...,...,...
207,37974184,1,249.99€,24999,samsung,https://www.varle.lt/mobilieji-telefonai/samsu...,mobilieji-telefonai,1733334394,1733334394
208,38371553,1,235.99€,23599,samsung,https://www.varle.lt/mobilieji-telefonai/samsu...,mobilieji-telefonai,1733334394,1733334394
209,39080345,1,355.99€,35599,samsung,https://www.varle.lt/mobilieji-telefonai/samsu...,mobilieji-telefonai,1733334394,1733334394
210,39090142,1,280.30€,28030,samsung,https://www.varle.lt/mobilieji-telefonai/telsa...,mobilieji-telefonai,1733334394,1733334394


In [ ]:
# Čia galima pradėti dirbti su DF, nuskaitytu iš DB.

mobilieji_telefonai = properties_of( "mobilieji_telefonai",
    engine="sqlite_file", db_path="./my_scrap_prop_of.db")

In [25]:
print('Atkuriamas DF iš properties_of()')
print(mobilieji_telefonai.export_to_df(['5g','price']))

Atkuriamas DF iš properties_of()
property_id object_id 5g     price
0            14511169  1   729.99€
1            14522358  1   709.99€
2            15087466  1   439.99€
3            15124168  1   511.38€
4            15124169  1   689.99€
..                ... ..       ...
207          37974184  1   249.99€
208          38371553  1   235.99€
209          39080345  1   355.99€
210          39090142  1   280.30€
211          40650587  1  1149.99€

[212 rows x 3 columns]


In [26]:
items_df=mobilieji_telefonai.export_to_df()

In [30]:
items_df

property_id,object_id,5g,price,price_in_cents,producer,url,type,create,update
0,14511169,1,729.99€,72999,samsung,https://www.varle.lt/mobilieji-telefonai/samsu...,mobilieji-telefonai,1733334394,1733334394
1,14522358,1,709.99€,70999,samsung,https://www.varle.lt/mobilieji-telefonai/isman...,mobilieji-telefonai,1733334394,1733334394
2,15087466,1,439.99€,43999,samsung,https://www.varle.lt/mobilieji-telefonai/samsu...,mobilieji-telefonai,1733334394,1733334394
3,15124168,1,511.38€,51138,samsung,https://www.varle.lt/mobilieji-telefonai/samsu...,mobilieji-telefonai,1733334394,1733334394
4,15124169,1,689.99€,68999,samsung,https://www.varle.lt/mobilieji-telefonai/samsu...,mobilieji-telefonai,1733334394,1733334394
...,...,...,...,...,...,...,...,...,...
207,37974184,1,249.99€,24999,samsung,https://www.varle.lt/mobilieji-telefonai/samsu...,mobilieji-telefonai,1733334394,1733334394
208,38371553,1,235.99€,23599,samsung,https://www.varle.lt/mobilieji-telefonai/samsu...,mobilieji-telefonai,1733334394,1733334394
209,39080345,1,355.99€,35599,samsung,https://www.varle.lt/mobilieji-telefonai/samsu...,mobilieji-telefonai,1733334394,1733334394
210,39090142,1,280.30€,28030,samsung,https://www.varle.lt/mobilieji-telefonai/telsa...,mobilieji-telefonai,1733334394,1733334394


In [28]:
import time

items_df['type'] = 'mobilieji-telefonai'
items_df['create'] = round(time.time())
items_df['update'] = round(time.time())

In [32]:
print('Iš DF kuriamas properties_of() ir įrašomas į DB.')

Iš DF kuriamas properties_of() ir įrašomas į DB.


In [33]:

varle_items_prop = properties_of( "varle_items",
    engine="sqlite_file", db_path="./my_scrap_prop_of.db")


In [43]:

varle_items_prop.import_from_df(items_df,id='object_id')


In [44]:

varle_items_prop.export_to_df().head(7)

property_id,object_id,5g,create,price,price_in_cents,producer,type,update,url
0,14511169,1,1733334394,729.99€,72999,samsung,mobilieji-telefonai,1733334394,https://www.varle.lt/mobilieji-telefonai/samsu...
1,14522358,1,1733334394,709.99€,70999,samsung,mobilieji-telefonai,1733334394,https://www.varle.lt/mobilieji-telefonai/isman...
2,15087466,1,1733334394,439.99€,43999,samsung,mobilieji-telefonai,1733334394,https://www.varle.lt/mobilieji-telefonai/samsu...
3,15124168,1,1733334394,511.38€,51138,samsung,mobilieji-telefonai,1733334394,https://www.varle.lt/mobilieji-telefonai/samsu...
4,15124169,1,1733334394,689.99€,68999,samsung,mobilieji-telefonai,1733334394,https://www.varle.lt/mobilieji-telefonai/samsu...
5,15154304,1,1733334394,439.99€,43999,samsung,mobilieji-telefonai,1733334394,https://www.varle.lt/mobilieji-telefonai/telek...
6,15154309,1,1733334394,629.99€,62999,samsung,mobilieji-telefonai,1733334394,https://www.varle.lt/mobilieji-telefonai/mobil...
